# Assignment 1 Group no. 10
### Project members: 
Veronika Cucorova <cucorova@kth.se>
Tim Roelofs <tjtro@kth.se>
Léo Vuylsteker <leov@kth.se>

### Declaration:
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy and pandas may be used in the solution for this assignment.

### Instructions
All assignments starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of assignments starting with 
number 2 below, then the assignment will receive 2 points (in total).

It is highly recommended that you do not develop the code directly within the notebook
but that you copy the comments and test cases to your regular development environment
and only when everything works as expected, that you paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).


## Load NumPy and pandas

In [1]:
import numpy as np
import pandas as pd

## 1a. Create and apply normalization

In [2]:
# Insert the functions create_normalization and apply_normalization below (after the comments)
#
# Input to create_normalization:
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
# normalizationtype: "minmax" (default) or "zscore"
#
# Output from create_normalization:
# df: a new dataframe, where each numeric value in a column has been replaced by a normalized value
# normalization: a mapping (dictionary) from each column name to a triple, consisting of
#                ("minmax",min_value,max_value) or ("zscore",mean,std)
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
# Hint 2: Consider columns of type "float" or "int" only (and which are not labeled "CLASS" or "ID"),
#         the other columns should remain unchanged
# Hint 3: Take a close look at the lecture slides on data preparation


def create_normalization(df, normalizationtype):
    #   we need to do the modifications on a deep copy of the dataframe 
    deep_copy_df = df.copy(deep=True)
    if normalizationtype == 'minmax':
        #   define the min and max function on a dataframe
        function1 = lambda dataframe: dataframe.min()
        function2 = lambda dataframe: dataframe.max()
    elif normalizationtype == 'zscore':
        #   define the mean and std function on a dataframe
        function1 = lambda dataframe: dataframe.mean()
        function2 = lambda dataframe: dataframe.std()
    else:
        #   if the keyword is not recognize, we make sure that the programme raises an error to indicate
        #   where the problem is
        raise ValueError('The normalizationtype variable has not been recognized. Please use either minmax '
                         'or zscore.')
    #   we remove the CLASS and ID columns, which have special meanings, from the working dataframe
    wrk_df = deep_copy_df[deep_copy_df.columns.difference(['CLASS', 'ID'])]
    #   we select the columns with contain numeric types
    wrk_df = wrk_df.select_dtypes(np.number)
    dict = {}
    for key in wrk_df.columns:
        #   we add the column name to the dictionary keys along with the corresponding normalization
        dict[key] = (normalizationtype, function1(wrk_df[key]), function2(wrk_df[key]))
        if normalizationtype == 'minmax':
            #   we apply the corresponding min-max normalization to the column
            deep_copy_df[key] = deep_copy_df[key].map(
                lambda x: (x-function1(wrk_df[key]))/(function2(wrk_df[key] - function1(wrk_df[key])))
            )
        else:
            #   we apply the corresponding z-normalization to the column
            deep_copy_df[key] = deep_copy_df[key].map(
                lambda x: (x-function1(wrk_df[key]))/function2(wrk_df[key])
            )
    return deep_copy_df, dict


# Input to apply_normalization:
# df: a dataframe
# normalization: a mapping (dictionary) from column names to triples (see above)
#
# Output from apply_normalization:
# df: a new dataframe, where each numerical value has been normalized according to the mapping
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
# Hint 2: For minmax-normalization, you may consider to limit the output range to [0,1]


def apply_normalization(df, normalization):
    #   we need to do the modifications on a deep copy of the dataframe 
    deep_copy_df = df.copy(deep=True)
    for key in normalization.keys():
        normalizationtype = normalization[key][0]
        if normalizationtype == 'minmax':
            min = normalization[key][1]
            max = normalization[key][2]
            #   we apply the corresponding min-max normalization to the column
            deep_copy_df[key] = deep_copy_df[key].map(
                lambda x: (x-min)/(max-min)
            )
        elif normalizationtype == 'zscore':
            #   we apply the corresponding z-normalization to the column
            mean = normalization[key][1]
            std = normalization[key][2]
            deep_copy_df[key] = deep_copy_df[key].map(
                lambda x: (x-mean)/std
            )
        else:
            #   if the keyword is not recognize, we make sure that the programme raises an error to indicate
            #   where the problem is
            raise ValueError('The normalizationtype variable has not been recognized. Please use either'
                             ' minmax or zscore.')
    return deep_copy_df


In [17]:
# Test your code (leave this part unchanged)

glass_train_df = pd.read_csv("..\..\glass_train.txt")

glass_test_df = pd.read_csv("..\..\glass_test.txt")

glass_train_norm, normalization = create_normalization(glass_train_df, normalizationtype="minmax")
print("normalization:\n")
for f in normalization:
    print("{}:{}".format(f,normalization[f]))
print(glass_train_df)
print(glass_train_norm)

glass_test_norm = apply_normalization(glass_test_df,normalization)
print("\nglass_test_norm:\n")
print(glass_test_norm)


normalization:

Al:('minmax', 0.29, 3.04)
Ba:('minmax', 0.0, 3.15)
Ca:('minmax', 5.43, 14.68)
Fe:('minmax', 0.0, 0.37)
K:('minmax', 0.0, 6.21)
Mg:('minmax', 0.0, 4.49)
Na:('minmax', 10.73, 15.79)
RI:('minmax', 1.51131, 1.53125)
Si:('minmax', 69.81, 75.18)
      ID       RI     Na    Mg    Al     Si     K     Ca    Ba    Fe  CLASS
0    202  1.51653  11.95  0.00  1.19  75.18  2.70   8.93  0.00  0.00      7
1    124  1.51707  13.48  3.48  1.71  72.52  0.62   7.99  0.00  0.00      2
2    152  1.52127  14.32  3.90  0.83  71.50  0.00   9.49  0.00  0.00      3
3    197  1.51556  13.87  0.00  2.54  73.23  0.14   9.41  0.81  0.01      7
4    144  1.51709  13.00  3.47  1.79  72.72  0.66   8.18  0.00  0.00      2
5     99  1.51689  12.67  2.88  1.71  73.21  0.73   8.54  0.00  0.00      2
6    207  1.51645  14.94  0.00  1.87  73.11  0.00   8.67  1.38  0.00      7
7     36  1.51567  13.29  3.45  1.21  72.74  0.56   8.57  0.00  0.00      1
8     50  1.51898  13.58  3.35  1.23  72.08  0.59   8.91  0.

### Comment on assumptions, things that do not work properly, etc.


## 1b. Create and apply imputation

In [57]:
# Insert the functions create_imputation and apply_imputation below (after the comments)
#
# Input to create_imputation:
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
#
# Output from create_imputation:
# df: a new dataframe, where each missing numeric value in a column has been replaced by the mean of that column 
#     and each missing categoric value in a column has been replaced by the mode of that column
# imputation: a mapping (dictionary) from column name to value that has replaced missing values
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
# Hint 2: Handle columns of type "float" or "int" only (and which are not labeled "CLASS" or "ID") in one way
#         and columns of type "object" and "category" in other ways
# Hint 3: Consider using the pandas functions mean() and mode() respectively, as well as fillna
# Hint 4: In the rare case of all values in a column being missing, replace numeric values with 0,
#         object values with "" and category values with the first category (cat.categories[0])  


def create_imputation(df):
    wrk_df = df.copy()
    dict = {}
    columns = wrk_df.columns.difference(['CLASS', 'ID '])
    for key in columns:
        #treat column as numeric
        if (wrk_df[key].dtypes == np.float64 or wrk_df[key].dtypes == np.int64):
            #first thing that came to my mind on checking if all values are null
            if (wrk_df[key].isnull().sum() == len(wrk_df[key])):
                repl_val = 0
            else:
                repl_val = wrk_df[key].mean()          
        #treat value as categorical
        elif (wrk_df[key].dtypes == np.bool or wrk_df[key].dtypes.name == 'category'):
            if (wrk_df[key].isnull().sum() == len(wrk_df[key])):
                repl_val = wrk_df[key].categories[0]
            else: 
                repl_val = wrk_df[key].mode()[0]
        #treat value as a string
        elif (wrk_df[key].dtypes == np.object):
            if (wrk_df[key].isnull().sum() == len(wrk_df[key])):
                repl_val = ""
            else: 
                repl_val = wrk_df[key].mode()[0] #mode can return multiple things
        else: 
            raise ValueError('Unknown column type.')

        dict[key] = (repl_val)
        wrk_df[key].fillna(repl_val, inplace = True)
    return wrk_df, dict
    

#
# Input to apply_imputation:
# df: a dataframe
# imputation: a mapping (dictionary) from column name to value that should replace missing values
#
# Output from apply_imputation:
# df: a new dataframe, where each missing value has been replaced according to the mapping
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
# Hint 2: Consider using fillna

def apply_imputation(df, repl_vals):
    wrk_df = df.copy()
    for key in repl_vals:
        wrk_df[key].fillna(repl_vals[key])
    return wrk_df





In [58]:
# Test your code (leave this part unchanged)

anneal_train_df = pd.read_csv("anneal_train.txt")
anneal_test_df = pd.read_csv("anneal_test.txt")

anneal_train_imp, imputation = create_imputation(anneal_train_df)
anneal_test_imp = apply_imputation(anneal_test_df,imputation)

print("Imputation:\n")
for f in imputation:
    print("{}:{}".format(f,imputation[f]))

print("\nNo. of replaced missing values in training data:\n{}".format(anneal_train_imp.count()-anneal_train_df.count()))
print("\nNo. of replaced missing values in test data:\n{}".format(anneal_test_imp.count()-anneal_test_df.count()))



Imputation:

bc:Y
bf:Y
bl:Y
blue-bright-varn-clean:B
bore:35.18930957683742
bt:Y
bw/me:B
carbon:3.859688195991091
cbond:Y
chrom:C
condition:S
corr:0
enamelability:1.7142857142857142
exptl:0
family:TN
ferro:Y
formability:2.2517482517482517
hardness:13.084632516703786
jurofm:0
len:1229.293986636971
lustre:Y
m:0
marvi:0
non-ageing:N
oil:Y
p:0
packing:3.0
phos:P
product-type:C
s:0
shape:SHEET
steel:A
strength:26.302895322939868
surface-finish:P
surface-quality:E
temper_rolling:T
thick:1.1911937639198227
width:769.4917594654789

No. of replaced missing values in training data:
family                    382
product-type                0
steel                      43
carbon                      0
hardness                    0
temper_rolling            374
condition                 160
formability               163
strength                    0
non-ageing                391
surface-finish            444
surface-quality           128
enamelability             442
bc                        448
b

### Comment on assumptions, things that do not work properly, etc.

## 1c. Create and apply discretization

In [33]:
# Insert the functions create_bins and apply_bins below
#
# Input to create_bins:
# df: a dataframe
# nobins: no. of bins (default = 10)
# bintype: either "equal-width" (default) or "equal-size" 
#
# Output from create_bins:
# df: a new dataframe, where each numeric feature value has been replaced by a categoric (corresponding to some bin)
# binning: a mapping (dictionary) from column name to bins (threshold values for the bin)
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
# Hint 2: Discretize columns of type "float" or "int" only (and which are not labeled "CLASS" or "ID")
# Hint 3: Consider using pd.cut and pd.qcut respectively, with labels=False, retbins=True and duplicates="drop"
#         (the last option will avoid errors when not enough bins can be created)
# Hint 4: Set all columns in the new dataframe to be of type "category"
# Hint 5: Set the categories of the discretized features to be [0,...,nobins-1]
# Hint 6: Change the first and the last element of each binning to -np.inf and np.inf respectively 
#
# Input to apply_bins:
# df: a dataframe
# binning: a mapping (dictionary) from column name to bins (threshold values for the bin)
#
# Output from apply_bins:
# df: a new dataframe, where each numeric feature value has been replaced by a categoric (corresponding to some bin)
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
# Hint 2: Consider using pd.cut 
# Hint 3: Set all columns in the new dataframe to be of type "category"
# Hint 4: Set the categories of the discretized features to be [0,...,nobins-1]
#



In [34]:
# Test your code  (leave this part unchanged)

glass_train_df = pd.read_csv("..\..\glass_train.txt")

glass_test_df = pd.read_csv("..\..\glass_test.txt")

glass_train_disc, binning = create_bins(glass_train_df,nobins=10,bintype="equal-size")
print("binning:\n")
for f in binning:
    print("{}:{}".format(f,binning[f]))

glass_test_disc = apply_bins(glass_test_df,binning)
print("\nglass_test_disc:\n")
glass_test_disc


binning:

RI:[    -inf 1.515896 1.51618  1.516516 1.516866 1.51753  1.517902 1.518618
 1.520114 1.521846      inf]
Na:[  -inf 12.73  12.872 13.    13.222 13.38  13.492 13.794 14.198 14.82
    inf]
Mg:[ -inf 1.82  3.188 3.41  3.476 3.55  3.61  3.728   inf]
Al:[ -inf 0.906 1.172 1.23  1.348 1.48  1.54  1.622 1.808 2.094   inf]
Si:[  -inf 71.756 72.196 72.388 72.72  72.79  72.966 73.06  73.208 73.372
    inf]
K:[ -inf 0.006 0.148 0.39  0.54  0.576 0.6   0.636 0.67    inf]
Ca:[  -inf  7.978  8.112  8.338  8.554  8.67   8.81   9.032  9.674 10.924
    inf]
Ba:[-inf 0.78  inf]
Fe:[ -inf 0.062 0.118 0.24    inf]

glass_test_disc:



,ID,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,CLASS
0,101,3,1,1,4,8,4,5,0,2,2
1,104,9,7,1,0,0,1,9,0,0,2
2,44,9,6,7,0,1,2,8,0,0,1
3,17,5,0,6,1,7,6,5,0,0,1
4,81,1,1,4,9,3,8,0,0,0,2
5,142,6,3,6,1,5,4,3,0,2,2
6,120,3,6,5,4,3,7,0,0,0,2
7,123,4,4,4,4,5,4,1,0,0,2
8,133,6,5,7,2,3,5,2,0,0,2
9,185,0,9,0,0,9,0,0,0,0,6


### Comment on assumptions, things that do not work properly, etc.

## 1d. Divide a dataset into a training and a test set

In [19]:
# Insert the function split below
#
# Input to split:
# df: a dataframe
# testfraction: a float in the range (0,1) (default = 0.5)
#
# Output from split:
# trainingdf: a dataframe consisting of a random sample of (1-testfraction) of the rows in df
# testdf: a dataframe consisting of the rows in df that are not included in trainingdf
#
# Hint: You may use np.random.permutation(df.index) to get a permuted list of indexes where a 
#       prefix corresponds to the test instances, and the suffix to the training instances 



In [20]:
# Test your code  (leave this part unchanged)

glass_df = pd.read_csv("glass.txt")

glass_train, glass_test = split(glass_df,testfraction=0.25)

print("Training IDs:\n{}".format(glass_train["ID"].values))

print("\nTest IDs:\n{}".format(glass_test["ID"].values))

print("\nOverlap: {}".format(set(glass_train["ID"]).intersection(set(glass_test["ID"]))))


Training IDs:
[ 55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214]

Test IDs:
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54]

Overlap: set()


### Comment on assumptions, things that do not work properly, etc.

## 1e. Calculate accuracy of a set of predictions

In [21]:
# Insert the function accuracy below
#
# Input to accuracy:
# df: a dataframe with class labels as column names and each row corresponding to
#     a prediction with estimated probabilities for each class
# correctlabels: an array (or list) of the correct class label for each prediction
#                (the number of correct labels must equal the number of rows in df)
#
# Output from accuracy:
# accuracy: the fraction of cases for which the predicted class label coincides with the correct label
#
# Hint: In case the label receiving the highest probability is not unique, you may
#       resolve that by picking the first (as ordered by the column names) or 
#       by randomly selecting one of the labels with highest probaility.



In [22]:
# Test your code  (leave this part unchanged)

predictions = pd.DataFrame({"A":[0.5,0.5,0.5,0.25,0.25],"B":[0.5,0.25,0.25,0.5,0.25],"C":[0.0,0.25,0.25,0.25,0.5]})
predictions


,A,B,C
0,0.50,0.50,0.00
1,0.50,0.25,0.25
2,0.50,0.25,0.25
3,0.25,0.50,0.25
4,0.25,0.25,0.50


In [23]:
correctlabels = ["B","A","B","B","C"]

accuracy(predictions,correctlabels) # Note that depending on how ties are resolved the accuracy may be 0.6 or 0.8

0.6

### Comment on assumptions, things that do not work properly, etc.

## 2a. Create and apply one-hot encoding

In [35]:
# Insert the functions create_one_hot and apply_one_hot below
#
# Input to create_one_hot:
# df: a dataframe
#
# Output from create_one_hot:
# df: a new dataframe, where each categoric feature has been replaced by a set of binary features 
#    (as many new features as there are possible values)
# one_hot: a mapping (dictionary) from column name to a set of categories (possible values for the feature)
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
# Hint 2: Consider columns of type "object" or "category" only (and which are not labeled "CLASS" or "ID")
# Hint 3: Consider creating new column names by merging the original column name and the categorical value
# Hint 4: Set all new columns to be of type "float"
# Hint 5: Do not forget to remove the original categoric feature
#
# Input to apply_one_hot:
# df: a dataframe
# one_hot: a mapping (dictionary) from column name to categories
#
# Output from apply_one_hot:
# df: a new dataframe, where each categoric feature has been replaced by a set of binary features
#
# Hint: See the above Hints



In [36]:
# Test your code  (leave this part unchanged)

tictactoe = pd.read_csv("tic-tac-toe.txt")

train_df, test_df = split(tictactoe) # Using your above function

new_train, one_hot = create_one_hot(train_df)

new_test = apply_one_hot(test_df,one_hot)
new_test

,CLASS,top-left-square-o,top-left-square-x,top-middle-square-b,top-middle-square-o,top-middle-square-x,top-right-square-b,top-right-square-o,top-right-square-x,middle-left-square-b,...,middle-right-square-x,bottom-left-square-b,bottom-left-square-o,bottom-left-square-x,bottom-middle-square-b,bottom-middle-square-o,bottom-middle-square-x,bottom-right-square-b,bottom-right-square-o,bottom-right-square-x
950,positive,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
483,positive,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
655,positive,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
699,positive,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
360,positive,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
329,positive,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
837,positive,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
97,positive,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
693,positive,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
498,positive,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


### Comment on assumptions, things that do not work properly, etc.

## 2b. Divide a dataset into a number of folds

In [26]:
# Insert the function folds below
#
# Input to folds:
# df: a dataframe
# nofolds: an integer greater than 1 (default = 10)
#
# Output from folds:
# folds: a list (of length = nofolds) dataframes consisting of random non-overlapping, 
#        approximately equal-sized subsets of the rows in df
#
# Hint: You may use np.random.permutation(df.index) to get a permuted list of indexes from which a 
#       prefix corresponds to the test instances, and the suffix to the training instances 



In [27]:
# Test your code  (leave this part unchanged)

glass_df = pd.read_csv("glass.txt")

glass_folds = folds(glass_df,nofolds=5)

fold_sizes = [len(f) for f in glass_folds]

print("Fold sizes:{}\nTotal no. instances: {}".format(fold_sizes,sum(fold_sizes)))

Fold sizes:[43, 43, 43, 43, 42]
Total no. instances: 214


### Comment on assumptions, things that do not work properly, etc.

## 2c. Calculate Brier score of a set of predictions

In [43]:
# Insert the function brier_score below
#
# Input to brier_score:
# df: a dataframe with class labels as column names and each row corresponding to
#     a prediction with estimated probabilities for each class
# correctlabels: an array (or list) of the correct class label for each prediction
#                (the number of correct labels must equal the number of rows in df)
#
# Output from brier_score:
# brier_score: the average square error of the predicted probabilties 
#
# Hint: Compare each predicted vector to a vector for each correct label, which is all zeros except 
#       for at the index of the correct class. The index can be found using np.where(df.columns==l)[0] 
#       where l is the correct label.



In [49]:
# Test your code  (leave this part unchanged)

predictions = pd.DataFrame({"A":[0.5,0.5,0.5,0.25,0.25],"B":[0.5,0.25,0.25,0.5,0.25],"C":[0.0,0.25,0.25,0.25,0.5]})

correctlabels = ["B","A","B","B","C"]

brier_score(predictions,correctlabels)

0.5

### Comment on assumptions, things that do not work properly, etc.

## 2d. Calculate AUC of a set of predictions

In [55]:
# Insert the function auc below
#
# Input to auc:
# df: a dataframe with class labels as column names and each row corresponding to
#     a prediction with estimated probabilities for each class
# correctlabels: an array (or list) of the correct class label for each prediction
#                (the number of correct labels must equal the number of rows in df)
#
# Output from auc:
# auc: the weighted area under ROC curve
#
# Hint 1: Calculate the binary AUC first for each class label c, i.e., treating the
#         predicted probability of this class for each instance as a score; the positive
#         instances are the ones belonging to class c and the negative instances the rest
# Hint 2: When calculating the binary AUC, first find the scores of the positive instances and then
#         the scores of the negative instances
# Hint 3: You may use a dictionary with a mapping from each score to an array of two numbers; 
#         the number of positive instances with this score and the number of negative instances with this score
# Hint 4: Created a (reversely) sorted (on the scores) list of pairs from the dictionary and
#         iterate over this to additively calculate the AUC
# Hint 5: For each pair in the above list, there are three cases to consider; the no. of true positives
#         (tp_i) is zero, the number of false positives (fp_i) (negatives) is zero, and both are non-zero
# Hint 6: Calculate the weighted AUC by summing the individual AUCs weighted by the relative
#         frequency of each class (as estimated from the correct labels)



In [56]:
# Test your code  (leave this part unchanged)

predictions = pd.DataFrame({"A":[0.9,0.9,0.6,0.55],"B":[0.1,0.1,0.4,0.45]})

correctlabels = ["A","B","B","A"]

auc(predictions,correctlabels)



0.375

In [57]:
predictions = pd.DataFrame({"A":[0.5,0.5,0.5,0.25,0.25],"B":[0.5,0.25,0.25,0.5,0.25],"C":[0.0,0.25,0.25,0.25,0.5]})

correctlabels = ["B","A","B","B","C"]

auc(predictions,correctlabels)

0.8499999999999999

### Comment on assumptions, things that do not work properly, etc.